In [1]:
import json
import math
import os
from datetime import date

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
from IPython.display import Image
from dotenv import load_dotenv
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

In [2]:
load_dotenv()
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
VERSION = date.today().strftime("%Y%m%d")

In [3]:
venue_id = '5756af9c498e47bf24348c6f'
url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results = requests.get(url).json()['response']

In [4]:
venue_data = results.get('venue', 0)
venue_data

0

In [5]:
v_prince = venue_data.get('price', 0)
if v_prince:
    print('tier:', v_prince['tier'])

AttributeError: 'int' object has no attribute 'get'

In [6]:
likes_data = venue_data['likes'].get('count', 0)
likes_data

TypeError: 'int' object is not subscriptable

In [7]:
rating_data = venue_data.get('rating', 0)
rating_data

AttributeError: 'int' object has no attribute 'get'

In [8]:
dislike_data = venue_data.get('dislike', 0)
dislike_data

AttributeError: 'int' object has no attribute 'get'

In [9]:
# 

In [10]:
minsk_venues_filtered = pd.read_csv('minsk_venues_filtered.csv')
minsk_venues_filtered.drop(columns=['Unnamed: 0'], inplace=True)
print(minsk_venues_filtered.shape)
minsk_venues_filtered.head()

(2187, 9)


,zip-code,neighborhood-latitude,neighborhood-longitude,venue-id,venue-name,venue-distance,venue-latitude,venue-longitude,venue-category
0,220001,53.887919,27.538119,4c8cf10555fba093fdb663ab,Баня № 7,192,53.889164,27.540151,Bath House
1,220001,53.887919,27.538119,55b29ffa498e519a9a539ac1,Hampton by Hilton,408,53.888951,27.544090,Hotel
2,220001,53.887919,27.538119,5b7d910167e5f2002cbaa676,Love Is Muffin,6,53.887879,27.538036,Café
3,220001,53.887919,27.538119,5756af9c498e47bf24348c6f,ціКава,358,53.888720,27.543410,Coffee Shop
4,220036,53.895040,27.520366,53653dee498ec5bdccef681a,Renaissance Minsk Hotel,622,53.892496,27.528823,Hotel


In [11]:
tdf = pd.DataFrame(data=np.zeros((2187, 4), dtype=float), columns=['prince-tier', 'likes', 'rating', 'dislike'])
tdf.head()

,prince-tier,likes,rating,dislike
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [12]:
df_minsk_venus_upd = pd.concat([minsk_venues_filtered, tdf], axis=1)
df_minsk_venus_upd.head()

,zip-code,neighborhood-latitude,neighborhood-longitude,venue-id,venue-name,venue-distance,venue-latitude,venue-longitude,venue-category,prince-tier,likes,rating,dislike
0,220001,53.887919,27.538119,4c8cf10555fba093fdb663ab,Баня № 7,192,53.889164,27.540151,Bath House,0.0,0.0,0.0,0.0
1,220001,53.887919,27.538119,55b29ffa498e519a9a539ac1,Hampton by Hilton,408,53.888951,27.544090,Hotel,0.0,0.0,0.0,0.0
2,220001,53.887919,27.538119,5b7d910167e5f2002cbaa676,Love Is Muffin,6,53.887879,27.538036,Café,0.0,0.0,0.0,0.0
3,220001,53.887919,27.538119,5756af9c498e47bf24348c6f,ціКава,358,53.888720,27.543410,Coffee Shop,0.0,0.0,0.0,0.0
4,220036,53.895040,27.520366,53653dee498ec5bdccef681a,Renaissance Minsk Hotel,622,53.892496,27.528823,Hotel,0.0,0.0,0.0,0.0


In [18]:
df_minsk_venus_upd = pd.read_csv('df_minsk_venus_upd.csv')

In [19]:
test_dtf = df_minsk_venus_upd['venue-id'][11:]

In [20]:
def get_venus_details():
    for index, venue_id in enumerate(test_dtf):
        try:
            url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
            results = requests.get(url).json()['response']
            venue = results.get('venue', None)

            if venue:
                v_prince = venue.get('price', 0)
                if v_prince:
                    vp_tier = v_prince['tier']
                    # vp_message = v_prince['message']
                    # print(vp_message, 'tier:', vp_tier)
                else:
                    vp_tier = 0

                likes = venue['likes'].get('count', 0)
                rating = venue.get('rating', 0)
                dislike = venue.get('dislike', 0)
                if not dislike:
                    dislike = 0
                else:
                    print("dislike: %s" % dislike)
                    dislike = 1

                df_minsk_venus_upd.at[index, 'prince-tier'] = vp_tier
                df_minsk_venus_upd.at[index, 'likes'] = likes
                df_minsk_venus_upd.at[index, 'rating'] = rating
                df_minsk_venus_upd.at[index, 'dislike'] = dislike
            else:
                print(index, 'Requests is None')
        except Exception as ex:
            print('Ex at: %s\n%s' % (index, ex))

In [21]:
# get_venus_details()

In [22]:
# df_minsk_venus_upd.to_csv('df_minsk_venus_upd.csv')

In [23]:
df_minsk_venus_upd

,Unnamed: 0,zip-code,neighborhood-latitude,neighborhood-longitude,venue-id,venue-name,venue-distance,venue-latitude,venue-longitude,venue-category,prince-tier,likes,rating,dislike
0,0,220001,53.887919,27.538119,4c8cf10555fba093fdb663ab,Баня № 7,192,53.889164,27.540151,Bath House,1.0,15.0,7.7,0.0
1,1,220001,53.887919,27.538119,55b29ffa498e519a9a539ac1,Hampton by Hilton,408,53.888951,27.544090,Hotel,1.0,123.0,8.2,0.0
2,2,220001,53.887919,27.538119,5b7d910167e5f2002cbaa676,Love Is Muffin,6,53.887880,27.538036,Café,1.0,6.0,7.4,0.0
3,3,220001,53.887919,27.538119,5756af9c498e47bf24348c6f,ціКава,358,53.888720,27.543410,Coffee Shop,1.0,7.0,7.7,0.0
4,4,220036,53.895040,27.520366,53653dee498ec5bdccef681a,Renaissance Minsk Hotel,622,53.892496,27.528823,Hotel,1.0,348.0,8.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182,2182,220141,53.938677,27.684619,5027750ae4b0d14d95d056fc,Баден,663,53.944484,27.686921,Shoe Store,0.0,0.0,0.0,0.0
2183,2183,220141,53.938677,27.684619,52bdba5d498e5e4c09b2fb84,Yves Rocher,773,53.935956,27.673765,Cosmetics Shop,0.0,0.0,0.0,0.0
2184,2184,220141,53.938677,27.684619,5333f04e11d2f0d39e7455da,Rieker,783,53.936630,27.673185,Shoe Store,0.0,0.0,0.0,0.0
2185,2185,220141,53.938677,27.684619,598433e54aa3f811584aab30,Мила,844,53.945940,27.688341,Cosmetics Shop,0.0,0.0,0.0,0.0


# TBA